In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from epoch_based_evolution import SearchSpace, Generation
import load_data

# Load Data

In [2]:
X_train, y_train, X_val, y_val, X_test, y_test = load_data.load_openml_dataset(dataset_id=334, scaling=True, random_seed=None, return_as='tensor')
input_size, output_size = load_data.get_tensor_sizes(X_train, y_train)

Class column is not numeric. Applying LabelEncoder.
Data loaded successfully! as <class 'torch.Tensor'>
Training data shape: torch.Size([384, 6])


In [4]:
# Load the CIFAR-10
X_train, y_train, X_val, y_val, X_test, y_test = load_data.load_cifar10_data(cifar_path='../CIFAR-10', return_as='tensor')
input_size, output_size = load_data.get_tensor_sizes(X_train, y_train)

Data loaded succesfully! as <class 'torch.Tensor'>
Training data shape: torch.Size([40000, 3, 32, 32])


In [7]:
search_space = SearchSpace(
    input_size=input_size,
    output_size=output_size,          
    # min_layers=2,           # Minimum number of hidden layers
    # max_layers=5,           # Maximum number of hidden layers
    # min_neurons=16,         # Minimum neurons per layer
    # max_neurons=256,        # Maximum neurons per layer
    # activation_fns=[nn.ReLU, nn.LeakyReLU],  # Activation functions to sample
    # dropout_rates=[0, 0.1, 0.2],             # Dropout rates to sample
    # min_learning_rate=0.0001,                # Minimum learning rate
    # max_learning_rate=0.01,                  # Maximum learning rate
    # random_seeds=[42, 13, 2024],             # Random seeds for reproducibility
    # min_batch_size=32,                       # Minimum batch size
    # max_batch_size=512                       # Maximum batch size
)


In [8]:
architecture = search_space.sample_architecture()
batch_size = architecture['batch_size']
model = search_space.create_model(architecture)
print(model)

DynamicNN(
  (network): Sequential(
    (0): Linear(in_features=3072, out_features=123, bias=True)
    (1): Sigmoid()
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=123, out_features=357, bias=True)
    (4): Sigmoid()
    (5): Dropout(p=0.1, inplace=False)
    (6): Linear(in_features=357, out_features=246, bias=True)
    (7): Sigmoid()
    (8): Dropout(p=0.1, inplace=False)
    (9): Linear(in_features=246, out_features=10, bias=True)
  )
  (criterion): CrossEntropyLoss()
)


In [9]:
# Create DataLoaders
train_dataset, train_loader = load_data.create_dataset_and_loader(X_train, y_train,
                                                        batch_size=batch_size)
val_dataset, val_loader = load_data.create_dataset_and_loader(X_val, y_val, 
                                                    batch_size=batch_size)
test_dataset, test_loader = load_data.create_dataset_and_loader(X_test, y_test,
                                                      batch_size=batch_size)

In [10]:
model.oe_train(train_loader)

(2.0328153898239134, 0.241425)

# Build the first generation

In [11]:
N_INDIVIDUALS = 50
first_gen = Generation(search_space, n_individuals=N_INDIVIDUALS)
# first_gen.generation

In [12]:
# TRAIN THE GEN
first_gen.train_generation(train_loader, 1)

In [13]:
first_gen.validate_generation(val_loader)

In [14]:
first_gen.generation[0]['val_loss']

2.1445950065612793

In [15]:
first_gen.get_worst_individuals(percentile_drop=15)
print(first_gen.worst_individuals)

[23, 3, 41, 34, 47, 11, 7]


In [16]:
first_gen.drop_worst_individuals()

In [17]:
len(first_gen.generation)

43

In [18]:
from epoch_based_evolution import run_generation

In [19]:
first_gen.generation

{0: {'model': DynamicNN(
    (network): Sequential(
      (0): Linear(in_features=3072, out_features=257, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.5, inplace=False)
      (3): Linear(in_features=257, out_features=481, bias=True)
      (4): ReLU()
      (5): Dropout(p=0.5, inplace=False)
      (6): Linear(in_features=481, out_features=46, bias=True)
      (7): ReLU()
      (8): Dropout(p=0.5, inplace=False)
      (9): Linear(in_features=46, out_features=92, bias=True)
      (10): ReLU()
      (11): Dropout(p=0.5, inplace=False)
      (12): Linear(in_features=92, out_features=374, bias=True)
      (13): ReLU()
      (14): Dropout(p=0.5, inplace=False)
      (15): Linear(in_features=374, out_features=98, bias=True)
      (16): ReLU()
      (17): Dropout(p=0.5, inplace=False)
      (18): Linear(in_features=98, out_features=462, bias=True)
      (19): ReLU()
      (20): Dropout(p=0.5, inplace=False)
      (21): Linear(in_features=462, out_features=10, bias=True)
    )
    (criter

In [21]:
number_of_epochs = 20
# create a new gen
generation = Generation(search_space, N_INDIVIDUALS)
for n_epoch in range(number_of_epochs + 1):
    print('\n-Epoch:', n_epoch)
    final_gen = run_generation(generation, train_loader, val_loader)
    # Extract val_loss values
    val_losses = [final_gen.generation[gen]['val_loss'] for gen in final_gen.generation]

    # Calculate the mean val_loss
    mean_val_loss = sum(val_losses) / len(val_losses)
    # Print the mean val_loss
    print("Mean val_loss:", mean_val_loss)

    # Extract val_Accuracy values
    val_accuracies = [final_gen.generation[gen]['val_acc'] for gen in final_gen.generation]

    # Calculate the mean val_loss
    mean_val_acc = sum(val_accuracies) / len(val_accuracies)
    # Print the mean val_loss
    print("Mean val_acc:", mean_val_acc)
    
    print('Survivor models:', len(final_gen.generation))




-Epoch: 0
Mean val_loss: 2.0908623748823647
Mean val_acc: 0.21348604651162792
Survivor models: 43

-Epoch: 1
Mean val_loss: 1.9850710687740427
Mean val_acc: 0.26521081081081077
Survivor models: 37

-Epoch: 2
Mean val_loss: 1.8977173089325432
Mean val_acc: 0.31438437499999994
Survivor models: 32

-Epoch: 3
Mean val_loss: 1.79371523364612
Mean val_acc: 0.34435000000000004
Survivor models: 28

-Epoch: 4
Mean val_loss: 1.7183056588649757
Mean val_acc: 0.3704708333333333
Survivor models: 24

-Epoch: 5
Mean val_loss: 1.654283877617972
Mean val_acc: 0.40369047619047616
Survivor models: 21

-Epoch: 6
Mean val_loss: 1.600517815981971
Mean val_acc: 0.4250777777777778
Survivor models: 18

-Epoch: 7
Mean val_loss: 1.5580468045830729
Mean val_acc: 0.44279374999999993
Survivor models: 16

-Epoch: 8
Mean val_loss: 1.532706580339159
Mean val_acc: 0.456
Survivor models: 14

-Epoch: 9
Mean val_loss: 1.507398098866145
Mean val_acc: 0.469325
Survivor models: 12

-Epoch: 10
Mean val_loss: 1.48513685333945